# Importing Packages

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras
import keras.models as M
import keras.layers as L
import tensorflow as tf
import keras.optimizers as opt
import keras.initializers as init
from column_encoder import *
import seaborn as sns
from numpy.random import randn
from numpy import hstack

# Reading Data

In [ ]:
data=pd.read_csv('../input/churn-for-bank-customers/churn.csv')

# Having a look and preprocessing data 

In [ ]:
data.head()

In [ ]:
# In this data the first three columns are not needed for the model so we will drop them
data.drop(columns=data.columns[:4],axis=1,inplace=True)

In [ ]:
# Using column encoder to transform categorical data to numerics
fit(data,['Geography','Gender'])
transform(data)

In [ ]:
# Having another look at the data
data.head()

In [ ]:
# Let's see the composition of the exited customers
sns.countplot(x='Exited',data=data)

# Making models and some functions

In [ ]:
# Since we have a lot less data for 1 let's make synthetic data for 1 in this file :)
def make_determinator_model(input_shape=9):
    model=M.Sequential(name='Determinator_Model')
    model.add(L.Dense(1000,input_dim=input_shape,activation='relu'))
    model.add(L.Dense(100,'relu'))
    model.add(L.Dense(1,'sigmoid'))
    model.compile(loss='binary_crossentropy',metrics=['accuracy'],optimizer='adam')
    return model

In [ ]:
# Make a function to create random latent dimension points for us
def make_random_latent(latent_dim=6,n=100):
    X=randn(latent_dim*n)
    X=np.absolute(X)*-10
    X=X.reshape((n,latent_dim))
    return X

In [ ]:
# Making Generator model Function for creating samples for class 1
def make_generator_model(input_shape,output_shape=9):
    model=M.Sequential(name="Generator_Model")
    model.add(L.Dense(1000,activation='relu',input_dim=input_shape))
    model.add(L.Dense(output_shape,activation='linear'))
    return model

In [ ]:
# Making Fake Samples 
def make_fake_samples(generator,latent_dim,n):
    fake=make_random_latent(latent_dim,n)
    predictions=generator.predict(fake)
    y=np.ones((n,1))
    return predictions,y

In [ ]:
# Making GAN model Function
def make_gan_model(generator,discriminator):
    discriminator.trainable=False
    model=M.Sequential(name='GAN_Model')
    model.add(generator)
    model.add(discriminator)
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    return model

In [ ]:
# Making a function to get the data from the dataframe
def get_data(data,target='Exited',batch_size=128):
    X=data.drop(target,axis=1).values
    y=data[target].values
    random_list=np.random.choice(len(data),batch_size)
    X=X[random_list]
    y=y[random_list]
    return X,y

In [ ]:
# Making a function to train the model
def train_model(g_model,d_model,gan_model,data,epochs=1000,n_counter=200,latent_dim=16,n=100,batch_size=128):
    for i in range(epochs):
        X,y=get_data(data,batch_size=128)
        d_model.train_on_batch(X,y)
        fake_x=make_random_latent(latent_dim,n)
        fake_y=np.ones((n,1))
        gan_model.train_on_batch(fake_x,fake_y)
        if i%n_counter==0:
            fake_x2=make_random_latent(latent_dim,n)
            fake_y2=np.ones((n,1))
            loss,acc=gan_model.evaluate(fake_x2,fake_y2,verbose=0)
            print('Epoch number is :',i,' The accuracy of the gan model for now is :',acc,'  Loss of the gan model is :',loss)
        

# Calling the functions and having a look at the models

In [ ]:
# Let's make the models now
latent_dim=16
d_model=make_determinator_model()
g_model=make_generator_model(latent_dim,output_shape=9)
gan_model=make_gan_model(g_model,d_model)

In [ ]:
# Let's look at the summary of the models
d_model.summary()

In [ ]:
# Let's look at the summary of the generator model
g_model.summary()

In [ ]:
# Sumamry of the gan model
gan_model.summary()

In [ ]:
# Calling the train model which will train the model
train_model(g_model,d_model,gan_model,data,latent_dim=latent_dim)

In [ ]:
# The model has worked perfectly . Let's have a look at the random samples that we have made now
test_data,y=make_fake_samples(g_model,16,4)

In [ ]:
# Let's try predictions on our already made model
d_model.predict_classes(test_data)

# The Samples might not be perfect but they are able to fool our model perfectly . The better the discriminator model the better the random fake samples will be too :)

## Reference taken from https://machinelearningmastery.com/how-to-develop-a-generative-adversarial-network-for-a-1-dimensional-function-from-scratch-in-keras/

# Thank you